# Velocity metrics
Idea is to measure movement of platform and to attempt to find largest changes in acceleration / velocity to describe sudden movements of platform.

In [5]:
# Import statements
from imageio import imread, imwrite
from pylab import *
from skimage.util import img_as_ubyte, img_as_float
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
from matplotlib.pyplot import imshow
from copy import copy
import os
import sys
import pandas

In [4]:
# Import local code
bachelorpath = os.path.normpath(os.getcwd() + os.sep + os.pardir)
scriptpath = bachelorpath + '/scripts/'
sys.path.insert(0, scriptpath)
import BP_ratio as bp
import dataset_plotting as dp
import motion_blur as mb 

## Use pandas to read IMU csv files and apply function to find metrics